# IMPORT LIBRARY

In [1]:
!pip install underthesea
!pip install sentencepiece
# !pip install transformers==4.44.2
!pip install einops
!pip install einops_exts
!pip install pandas
# !pip install torch==2.4.0
# !pip install torchvision==0.19.0
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


In [2]:
from underthesea import word_tokenize,text_normalize
import torch
from transformers import AutoTokenizer,get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import pandas as pd
import json
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import re
import sentencepiece

In [3]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel, AutoImageProcessor
from transformers import CLIPVisionModel, CLIPVisionConfig

import torch
from torchvision.io import read_image
import open_clip

In [4]:
import transformers
transformers.logging.set_verbosity_error()

In [5]:
sss = 18 + 19
torch.manual_seed(sss)
np.random.seed(sss)

# TEXT PRE-PROCESSING

In [6]:
def convert_unicode(text):
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  char1252 = char1252.split('|')
  charutf8 = charutf8.split('|')

  dic = {}
  for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
  return re.sub(
      r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
      lambda x: dic[x.group()], text
)


In [7]:
import unicodedata

class TextNormalize:
    def __init__(self):
        self.vowels_to_ids = {}
        self.vowels_table = [
            ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
            ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
            ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
            ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
            ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
            ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
            ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
            ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'o'],
            ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
            ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
            ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
            ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
        ]
        pass

    def createVowelsTable(self):
        """Create Vowels Table"""
        for i in range(len(self.vowels_table)):
            for j in range(len(self.vowels_table[i]) - 1):
                self.vowels_to_ids[self.vowels_table[i][j]] = (i, j)

    def IsValidVietnameseWord(self,word):
        """Nguyên âm chỉ có thể đứng chung với nguyên âm. Một từ không thể có 2 nguyên âm cách nhau bởi 1 phụ âm"""
        chars = list(word)
        #nguyen am
        vowel_index = -1
        for i in range(len(chars)):
            idx_vowel_table = self.vowels_to_ids.get(chars[i],(-1,-1))[0]
            if idx_vowel_table != -1:
                if vowel_index == -1:
                    vowel_index = i
                else:
                    if i - vowel_index != 1:
                        return False
                    vowel_index = i
        return True

    def WordStandardized(self,word):
        """Standardize Word"""
        if not self.IsValidVietnameseWord(word):
            return word

        chars = list(word)
        vowel_indexes = []

        # tìm vị trí nguyên âm
        qu_or_gi = False
        thanh_dieu = 0
        for i in range(len(chars)):
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[i],(-1,-1))
            if vowel_table_row == -1 :
                continue
            # qu
            if vowel_table_row == 9:
                if i != 0 and chars[i-1] == 'q':
                    chars[i] = 'u'
                    qu_or_gi = True
            # gi
            elif vowel_table_row == 5:
                if i != 0 and chars[i-1] == 'g':
                    chars[i] = 'i'
                    qu_or_gi = True

            # có chứa thanh điệu
            if vowel_table_col != 0:
                thanh_dieu = vowel_table_col
                chars[i] = self.vowels_table[vowel_table_row][0]

            vowel_indexes.append(i)
        # 1 nguyên âm
        if len(vowel_indexes) == 1:
            c = chars[vowel_indexes[0]]
            chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
            return ''.join(chars)

        for idx_vowel in vowel_indexes:
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[idx_vowel],(-1,-1))
            #ê, ơ, ô
            if vowel_table_row == 4 or vowel_table_row == 7 or vowel_table_row == 8:
                c = chars[idx_vowel]
                chars[idx_vowel] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # kiểm tra qu và gi, 2-3 nguyên âm thì nguyên âm thứ 2 chứa dấu
            if qu_or_gi:
                if len(vowel_indexes) == 2 or len(vowel_indexes) == 3:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # 2 nguyên âm
            if len(vowel_indexes) == 2:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[0]]
                    chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            elif len(vowel_indexes) == 3:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

        return ''.join(chars)

    def normalize(self,text):


        #Chuyen sang viet thuong
        text = text.lower()
        text = unicodedata.normalize('NFC', text)

        # Rút gọn từ kéo dài
        text = re.sub(r'(\W)\1+',r'\1',text)

        # xóa các emoji dư thừa
#         emoji_pattern = re.compile(
#             "["
#             "\U0001F600-\U0001F64F"  # Emoticons
#             "\U0001F300-\U0001F5FF"  # Symbols & pictographs
#             "\U0001F680-\U0001F6FF"  # Transport & map symbols
#             "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
#             "\U00002500-\U00002BEF"  # Chinese char
#             "\U00002702-\U000027B0"
#             "\U00002702-\U000027B0"
#             "\U000024C2-\U0001F251"
#             "\U0001f926-\U0001f937"
#             "\U00010000-\U0010ffff"
#             "]+",
#             flags=re.UNICODE,
#         )
#         text = emoji_pattern.sub(r'',text) # no emoji
        # remove hastag
        text = re.sub("(@[A-Za-z0-9]+)|(#[0-9A-Za-z]+)","", text)

        # xóa space d
        text = re.sub(r"( )\1+",r'\1',text)
#         text = re.sub(r"[:)^@!`~%;?(\+\-\'\"]+",r'',text)
        text = text.replace("“","")
        text = text.replace("\n", " ")
        text = " ".join(text.split())
        return text

In [8]:
%ls

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_json('/kaggle/input/train-dsc2024/vimmsd-train.json',orient = 'index').reset_index()
data = data[data['label'] != 'not-sarcasm']
data['label'].value_counts()

label
multi-sarcasm    4224
image-sarcasm     442
text-sarcasm       77
Name: count, dtype: int64

In [17]:
train_data, dev_data = train_test_split(data, test_size=0.1, stratify=data['label'], random_state = sss)
train_data = train_data.reset_index()
dev_data = dev_data.reset_index()

In [10]:
normalize_class = TextNormalize()

In [19]:
train_data['caption'] = train_data['caption'].apply(lambda x: text_normalize(normalize_class.normalize(x)))
dev_data['caption'] = dev_data['caption'].apply(lambda x: text_normalize(normalize_class.normalize(x)))

In [20]:
train_data

,level_0,index,image,caption,label
0,9891,9891,bc723746ccc5124698bfb0c06b895ca761e6097c21c87c...,thần cồn jack trong bữa tiệc mừng vô địch prem...,multi-sarcasm
1,8311,8311,e592bb379b4517a2b1d3aee3f7af187d4a59cdd48a3030...,🔥 góc đêm hè : baekhyun khoe qua việt nam giảm...,multi-sarcasm
2,2613,2613,e427e0040ef3cccd6c7ba7555b3e6d91dc6aab6241bef6...,vì một cái tết vui vẻ . đây là giấy a4 dán trư...,multi-sarcasm
3,8307,8307,139d869d2d36b7d10bbab884723126a058c77315164736...,- kem -,image-sarcasm
4,3526,3526,252790c126b210c28a6f26808a440d302c0f88bfb03113...,cảm động tiếp viên hàng không tận tụy đưa từng...,multi-sarcasm
...,...,...,...,...,...
4263,2629,2629,11e5a85c54ede81aff3b4601a18a82ff3af2564589096b...,"- biển nó canh mài mà , chứ mài đâu có canh nó...",multi-sarcasm
4264,4995,4995,2a05fbd099194bcb091cc59b3c0a8d57a12f3916a41962...,"quỷ đầu đàn rực cháy , mu hạ gục manchester ci...",multi-sarcasm
4265,7605,7605,bd9a373ab7e13c893e6319c4f8c077cf3f87f79d603b0b...,lê phan sinh ngày 19 tháng 11 năm 2001 tại hà ...,image-sarcasm
4266,880,880,f282592bf2e1b4886fd1fef3d7fecf68fa54ca0c4076f0...,"khi bị gỡ mìn , cư dân trên đảo đã phản đối kị...",text-sarcasm


In [11]:
def remove_numbers_and_special_chars(text):
    # Use regular expression to remove numbers and special characters, keep letters with accent marks
    clean_text = re.sub(r'[^\w\s]', '', text)  # This removes special characters but keeps accent marks
    clean_text = re.sub(r'\d+', '', clean_text)  # This removes numbers
    clean_text = ' '.join([word for word in clean_text.split() if len(word) > 1])

    return clean_text


In [12]:
import os

ocr_text_dict = {}
ocr_files = os.listdir('/kaggle/input/ocr-text')
for file in ocr_files:
    textt = np.load('/kaggle/input/ocr-text/' + file, allow_pickle=True)[()]['pixel_values']
    ocr_text_dict[file.split('.')[0]] = remove_numbers_and_special_chars(text_normalize(normalize_class.normalize(textt)))
# ocr_text_dict

In [13]:
class ViMMDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, img_folder, vision_processor, max_len, ocr_text_dict):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.img_folder = img_folder
        self.vision_processor = vision_processor
        self.labels = ['image-sarcasm','text-sarcasm','multi-sarcasm']
        self.ocr_text_dict = ocr_text_dict
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        text = self.data.loc[idx,'caption']
        img_name = self.data.loc[idx,'image']
        img_file = os.path.join(self.img_folder,img_name)
        label = self.data.loc[idx,'label']
        label = self.labels.index(label)

        ocr_text = self.ocr_text_dict[img_name.split('.')[0]]
        ocr_tokenizer = self.tokenizer(ocr_text, max_length=self.max_len,truncation='only_first',padding='max_length', return_length=True, return_tensors='pt')
        
        ocr_text = " ".join(ocr_text.split(" ")[:80])
        tokenizer_seq_dict = self.tokenizer(f"<image> {ocr_text}", text, max_length=self.max_len,truncation='longest_first',padding='max_length', return_length=True, return_tensors='pt')
        text_tokenizer = self.tokenizer(text, max_length=self.max_len,truncation='only_first',padding='max_length', return_length=True, return_tensors='pt')
        
        input_ids = tokenizer_seq_dict['input_ids'][0]
        attention_mask = (input_ids != 1).long()

        text_input_ids = text_tokenizer['input_ids'][0]
        text_attention_mask = (text_input_ids != 1).long()

        ocr_input_ids = ocr_tokenizer['input_ids'][0]
        ocr_attention_mask = (ocr_input_ids != 1).long()

        image = Image.open(img_file)
        pixel_values = self.vision_processor(image).unsqueeze(0).unsqueeze(0)

        return input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, torch.tensor(label)

In [14]:
PRETRAINED_PATH = "susnato/ernie-m-base_pytorch"
VISION_PRETRAINED_PATH = 'facebook/dinov2-large'

In [15]:
# vision_processor = AutoImageProcessor.from_pretrained(VISION_PRETRAINED_PATH)

In [16]:
_, _, vision_processor = open_clip.create_model_and_transforms(
    'ViT-L-14-quickgelu',
    pretrained='dfn2b'
)

open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)
tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<image>"]}
    )


tokenizer_config.json:   0%|          | 0.00/219 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [28]:
train_set = ViMMDataset(train_data, tokenizer, '/kaggle/input/train-dsc2024/training-images/train-images', vision_processor, max_len = 256,ocr_text_dict=ocr_text_dict)
dev_set = ViMMDataset(dev_data, tokenizer, '/kaggle/input/train-dsc2024/training-images/train-images', vision_processor, max_len = 256,ocr_text_dict=ocr_text_dict)

In [29]:
train_set[0][1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
# for i in range(len(train_set)):
#     try:
#         t = train_set[i]
#     except:
#         print(i)

In [31]:
aa = 0
for i in train_set[aa][0]:
    print(tokenizer.convert_ids_to_tokens(i.item()),end=' ' )
print()

[CLS] <image> ▁trước ▁sau ▁buổi ▁tiệc ▁bu ôi ▁tiệc ▁a ce book ▁com f roll bong da ▁youtube com ▁troll bong da official [SEP] [SEP] ▁thần ▁c ồn ▁jack ▁trong ▁bữa ▁tiệc ▁mừng ▁vô ▁địch ▁premier ▁league : ▁v [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [93]:
from tqdm import tqdm
y = []
for i in tqdm(range(len(train_set))):
  y.append(train_set[i][-1].item())


100%|██████████| 4268/4268 [00:03<00:00, 1295.40it/s]


In [94]:
y[:20]

[2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [95]:
import numpy as np
from torch.utils.data.sampler import WeightedRandomSampler

counts = np.bincount(y)
labels_weights = 1. / counts
# labels_weights[-1] = labels_weights[-1]
weights = labels_weights[y]
print(counts,labels_weights)

[ 398   69 3801] [0.00251256 0.01449275 0.00026309]


In [59]:
# train_loader = torch.utils.data.DataLoader(train_set, batch_size = 2, sampler=WeightedRandomSampler(weights, len(weights)) )# )
dev_loader = torch.utils.data.DataLoader(dev_set, batch_size = 32)

# MODEL

In [18]:
import torch
import torch.nn as nn
import copy
import numpy as np
import math
import torch.nn.functional as F
from transformers import AutoModel
from torch.autograd import Variable


In [19]:
import torch
from einops import rearrange, repeat
from einops_exts import rearrange_many
from torch import einsum, nn

In [20]:
import os
mode = 'FP16'
os.environ["TORCH_CUDNN_V8_API_ENABLED"] = "1"
torch.backends.cuda.matmul.allow_tf32 = True if mode == 'TF32' else False
scaler = torch.cuda.amp.GradScaler(enabled=True if mode == 'FP16' else False)


/tmp/ipykernel_30/3556953468.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True if mode == 'FP16' else False)


In [21]:
class BertLayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(BertLayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

In [22]:
def exists(val):
    return val is not None

def FeedForward(dim, mult=4):
    inner_dim = int(dim * mult)
    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, inner_dim, bias=False),
        nn.GELU(),
        nn.Linear(inner_dim, dim, bias=False),
    )

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
class PerceiverAttention(nn.Module):
    def __init__(self, *, dim, dim_head=64, heads=8):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm_media = nn.LayerNorm(dim)
        self.norm_latents = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

    def forward(self, x, latents):
        """
        Args:
            x (torch.Tensor): image features
                shape (b, T, n1, D)
            latent (torch.Tensor): latent features
                shape (b, T, n2, D)
        """
        x = self.norm_media(x)
        latents = self.norm_latents(latents)

        h = self.heads

        q = self.to_q(latents)
        kv_input = torch.cat((x, latents), dim=-2)
        k, v = self.to_kv(kv_input).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b t n (h d) -> b h t n d", h=h)
        q = q * self.scale

        # attention
        sim = einsum("... i d, ... j d  -> ... i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h t n d -> b t n (h d)", h=h)
        return self.to_out(out)

In [25]:
class PerceiverResampler(nn.Module):
    def __init__(
        self,
        *,
        dim,
        depth=6,
        dim_head=64,
        heads=8,
        num_latents=64,
        max_num_media=None,
        max_num_frames=None,
        ff_mult=4,
    ):
        super().__init__()
        self.latents = nn.Parameter(torch.randn(num_latents, dim))
        self.frame_embs = (
            nn.Parameter(torch.randn(max_num_frames, dim))
            if exists(max_num_frames)
            else None
        )
        self.media_time_embs = (
            nn.Parameter(torch.randn(max_num_media, 1, dim))
            if exists(max_num_media)
            else None
        )

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads),
                        FeedForward(dim=dim, mult=ff_mult),
                    ]
                )
            )

        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): image features
                shape (b, T, F, v, D)
        Returns:
            shape (b, T, n, D) where n is self.num_latents
        """
        b, T, F, v = x.shape[:4]

        # frame and media time embeddings
        if exists(self.frame_embs):
            frame_embs = repeat(self.frame_embs[:F], "F d -> b T F v d", b=b, T=T, v=v)
            x = x + frame_embs
        x = rearrange(
            x, "b T F v d -> b T (F v) d"
        )  # flatten the frame and spatial dimensions
        if exists(self.media_time_embs):
            x = x + self.media_time_embs[:T]

        # blocks
        latents = repeat(self.latents, "n d -> b T n d", b=b, T=T)
        for attn, ff in self.layers:
            latents = attn(x, latents) + latents
            latents = ff(latents) + latents
        return self.norm(latents)

In [26]:
# gated cross attention
class MaskedCrossAttention(nn.Module):
    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        only_attend_immediate_media=True,
    ):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim_visual, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        # whether for text to only attend to immediate preceding image, or all previous images
        self.only_attend_immediate_media = only_attend_immediate_media

    def forward(self, x, media, media_locations=None, use_cached_media=False):
        """
        Args:
            x (torch.Tensor): text features
                shape (B, T_txt, D_txt)
            media (torch.Tensor): image features
                shape (B, T_img, n, D_img) where n is the dim of the latents
            media_locations: boolean mask identifying the media tokens in x
                shape (B, T_txt)
            use_cached_media: bool
                If true, treat all of x as if they occur after the last media
                registered in media_locations. T_txt does not need to exactly
                equal media_locations.shape[1] in this case
        """

        if not use_cached_media:
            assert (
                media_locations.shape[1] == x.shape[1]
            ), f"media_location.shape is {media_locations.shape} but x.shape is {x.shape}"

        T_txt = x.shape[1]
        _, T_img, n = media.shape[:3]
        h = self.heads

        x = self.norm(x)

        q = self.to_q(x)
        media = rearrange(media, "b t n d -> b (t n) d")

        k, v = self.to_kv(media).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b n (h d) -> b h n d", h=h)

        q = q * self.scale

        sim = einsum("... i d, ... j d -> ... i j", q, k)

        if exists(media_locations):
            media_time = torch.arange(T_img, device=x.device) + 1

            if use_cached_media:
                # text time is set to the last cached media location
                text_time = repeat(
                    torch.count_nonzero(media_locations, dim=1),
                    "b -> b i",
                    i=T_txt,
                )
            else:
                # at each boolean of True, increment the time counter (relative to media time)
                text_time = media_locations.cumsum(dim=-1)

            # text time must equal media time if only attending to most immediate image
            # otherwise, as long as text time is greater than media time (if attending to all previous images / media)
            mask_op = torch.eq if self.only_attend_immediate_media else torch.ge

            text_to_media_mask = mask_op(
                rearrange(text_time, "b i -> b 1 i 1"),
                repeat(media_time, "j -> 1 1 1 (j n)", n=n),
            )
            sim = sim.masked_fill(~text_to_media_mask, -torch.finfo(sim.dtype).max)

        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        if exists(media_locations) and self.only_attend_immediate_media:
            # any text without a preceding media needs to have attention zeroed out
            text_without_media_mask = text_time == 0
            text_without_media_mask = rearrange(
                text_without_media_mask, "b i -> b 1 i 1"
            )
            attn = attn.masked_fill(text_without_media_mask, 0.0)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)")
        return self.to_out(out)

In [27]:
class GatedCrossAttentionBlock(nn.Module):
    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        ff_mult=4,
        only_attend_immediate_media=True,
    ):
        super().__init__()
        self.attn = MaskedCrossAttention(
            dim=dim,
            dim_visual=dim_visual,
            dim_head=dim_head,
            heads=heads,
            only_attend_immediate_media=only_attend_immediate_media,
        )
        self.ff = FeedForward(dim, mult=ff_mult)
        self.norm = nn.LayerNorm(dim)
    def forward(
        self,
        x,
        media,
        media_locations=None,
        use_cached_media=False,
    ):
        x = (
            self.attn(
                x,
                media,
                media_locations=media_locations,
                use_cached_media=use_cached_media,
            )
            + x
        )
        x = self.norm(self.ff(x) + x)

        return x

In [28]:
def apply_with_stopping_condition(
    module, apply_fn, apply_condition=None, stopping_condition=None, **other_args
):
    if stopping_condition(module):
        return
    if apply_condition(module):
        apply_fn(module, **other_args)
    for child in module.children():
        apply_with_stopping_condition(
            child,
            apply_fn,
            apply_condition=apply_condition,
            stopping_condition=stopping_condition,
            **other_args
        )

In [29]:
def extend_instance(obj, mixin):
    """Apply mixins to a class instance after creation"""
    base_cls = obj.__class__
    base_cls_name = obj.__class__.__name__
    obj.__class__ = type(
        base_cls_name, (mixin, base_cls), {}
    )  # mixin needs to go first for our forward() logic to work


def getattr_recursive(obj, att):
    """
    Return nested attribute of obj
    Example: getattr_recursive(obj, 'a.b.c') is equivalent to obj.a.b.c
    """
    if att == "":
        return obj
    i = att.find(".")
    if i < 0:
        return getattr(obj, att)
    else:
        return getattr_recursive(getattr(obj, att[:i]), att[i + 1 :])


def setattr_recursive(obj, att, val):
    """
    Set nested attribute of obj
    Example: setattr_recursive(obj, 'a.b.c', val) is equivalent to obj.a.b.c = val
    """
    if "." in att:
        obj = getattr_recursive(obj, ".".join(att.split(".")[:-1]))
    setattr(obj, att.split(".")[-1], val)



In [30]:
class FlamingoLayer(nn.Module):
    """
    FlamingoLayer is a wrapper around the GatedCrossAttentionBlock and DecoderLayer.
    """

    def __init__(
        self, gated_cross_attn_layer, encoder_layer, gradient_checkpointing=False
    ):
        super().__init__()
        self.gated_cross_attn_layer = gated_cross_attn_layer
        self.vis_x = None
        self.encoder_layer = encoder_layer
        self.media_locations = None
        if self.gated_cross_attn_layer is not None:
            self.gated_cross_attn_layer._use_gradient_checkpointing = (
                gradient_checkpointing
            )

    def is_conditioned(self) -> bool:
        """Check whether the layer is conditioned."""
        return self.vis_x is not None and self.media_locations is not None

    # Used this great idea from this implementation of Flamingo (https://github.com/dhansmair/flamingo-mini/)
    def condition_vis_x(self, vis_x):
        self.vis_x = vis_x

    def condition_media_locations(self, media_locations):
        self.media_locations = media_locations

    def condition_use_cached_media(self, use_cached_media):
        self.use_cached_media = use_cached_media

    def forward(
        self,
        *args,
        **encoder_layer_kwargs
    ):
        # Cross attention
        lang_x = encoder_layer_kwargs['hidden_states']
        attention_mask = encoder_layer_kwargs['attention_mask']

        if self.gated_cross_attn_layer is not None:
            if self.vis_x is None:
                raise ValueError("vis_x must be conditioned before forward pass")

            if self.media_locations is None:
                raise ValueError(
                    "media_locations must be conditioned before forward pass"
                )

            lang_x = self.gated_cross_attn_layer(
                lang_x,
                self.vis_x,
                media_locations=self.media_locations,
                use_cached_media=self.use_cached_media,
            )

        # Normal decoder layer
        lang_x = self.encoder_layer(
            lang_x, attention_mask, *args
        )
        return lang_x

In [31]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel

class FlamingoLMMixin(nn.Module):
    """
    Mixin to add cross-attention layers to a language model.
    """

    def set_encoder_layers_attr_name(self, encoder_layers_attr_name):
        self.encoder_layers_attr_name = encoder_layers_attr_name

    def _get_encoder_layers(self):
        return getattr_recursive(self, self.encoder_layers_attr_name)

    def _set_encoder_layers(self, value):
        setattr_recursive(self, self.encoder_layers_attr_name, value)

    def init_flamingo(
        self,
        media_token_id,
        lang_hidden_size,
        vis_hidden_size,
        cross_attn_every_n_layers,
        gradient_checkpointing,
    ):
        """
        Initialize Flamingo by adding a new gated cross attn to the decoder. Store the media token id for computing the media locations.
        """
        self.old_encoder_blocks = self._get_encoder_layers()
        self.gated_cross_attn_layers = nn.ModuleList(
            [
                GatedCrossAttentionBlock(
                    dim=lang_hidden_size, dim_visual=vis_hidden_size
                )
                if (layer_idx + 1) % cross_attn_every_n_layers == 0 and layer_idx != 0
                else None
                for layer_idx, _ in enumerate(self._get_encoder_layers())
            ]
        )
        self.init_flamingo_layers(gradient_checkpointing)
        self.media_token_id = media_token_id
        self.initialized_flamingo = True
        self._use_cached_vision_x = False

    def init_flamingo_layers(self, gradient_checkpointing):
        """
        Re initializes the FlamingoLayers.
        Propagates any changes made to self.gated_corss_attn_layers or self.old_encoder_blocks
        """
        self._set_encoder_layers(
            nn.ModuleList(
                [
                    FlamingoLayer(
                        gated_cross_attn_layer, encoder_layer, gradient_checkpointing
                    )
                    for gated_cross_attn_layer, encoder_layer in zip(
                        self.gated_cross_attn_layers, self.old_encoder_blocks
                    )
                ]
            )
        )

    def forward(self, input_ids, attention_mask, **kwargs):

        """Condition the Flamingo layers on the media locations before forward()"""
        if not self.initialized_flamingo:
            raise ValueError(
                "Flamingo layers are not initialized. Please call `init_flamingo` first."
            )
        media_locations = input_ids == self.media_token_id

        # if there are media already cached and we're generating and there are no media tokens in the input,
        # we'll assume that ALL input tokens should attend to the last previous media that is cached.
        # this is especially important for HF generate() compatibility, since generate() calls forward()
        # repeatedly one token at a time (with no media tokens).
        # without this check, the model would not attend to any images when generating (after the first token)
        use_cached_media_locations = (
            self._use_cached_vision_x
            and self.is_conditioned()
            and not media_locations.any()
        )

        for layer in self._get_encoder_layers():
            if not use_cached_media_locations:
                layer.condition_media_locations(media_locations)
            layer.condition_use_cached_media(use_cached_media_locations)

        # package arguments for the other parent's forward. since we don't know the order of the arguments,
        # make them all kwargs

        kwargs["input_ids"] = input_ids
        kwargs["attention_mask"] = attention_mask

        return super().forward(**kwargs)  # Call the other parent's forward method

    def is_conditioned(self) -> bool:
        """Check whether all decoder layers are already conditioned."""
        return all(l.is_conditioned() for l in self._get_encoder_layers())

    def clear_conditioned_layers(self):
        for layer in self._get_encoder_layers():
            layer.condition_vis_x(None)
            layer.condition_media_locations(None)
            layer.condition_use_cached_media(None)

In [32]:
class ClassificationHead(nn.Module):

    def __init__(
        self,
        input_dim,
        inner_dim,
        num_classes,
        pooler_dropout,
        pooler_classifier,
    ):
        super().__init__()
        self.pooler_classifier = pooler_classifier
        if pooler_classifier == "mlp":
            self.dense = nn.Linear(input_dim, inner_dim)
            self.activation_fn = nn.Tanh()
            self.dropout = nn.Dropout(p=pooler_dropout)
            self.out_proj = nn.Linear(inner_dim, num_classes)
        elif pooler_classifier == "linear":
            self.dropout = nn.Dropout(p=pooler_dropout)
            self.out_proj = nn.Linear(input_dim, num_classes)
        else:
            raise NotImplementedError

    def forward(self, features):
        if self.pooler_classifier == 'mlp':
            x = features
            x = self.dropout(x)
            x = self.dense(x)
            x = self.activation_fn(x)
            x = self.dropout(x)
            x = self.out_proj(x)
        elif self.pooler_classifier == 'linear':
            x = features
            x = self.dropout(x)
            x = self.out_proj(x)
        else:
            raise NotImplementedError
        return x


In [33]:
import torch
from einops import rearrange
from torch import nn
from transformers.modeling_outputs import CausalLMOutputWithPast
from torch.distributed.fsdp import (
    FullyShardedDataParallel as FSDP,
)

class Flamingo(nn.Module):
    def __init__(
        self,
        vision_encoder: nn.Module,
        lang_encoder: nn.Module,
        first_lang_encoder: nn.Module,
        ocr_lang_encoder: nn.Module,
        vis_dim: int,
        media_token_id: int,
        n_classes = 4,
        cross_attn_every_n_layers: int = 1,
        gradient_checkpointing: bool = False,
    ):
        """
        Args:
            vision_encoder (nn.Module): HF CLIPModel
            lang_encoder (nn.Module): HF causal language model
                vis_dim (int): Dimension of the visual features.
                Visual features are projected to match this shape along the last dimension.
            cross_attn_every_n_layers (int, optional): How often to apply cross attention after transformer layer. Defaults to 1.
        """
        super().__init__()

        self.media_token_id = media_token_id
        self.vis_dim = vis_dim
        if hasattr(lang_encoder.config, "d_model"):
            self.lang_dim = lang_encoder.config.d_model  # mpt uses d_model
        else:
            self.lang_dim = lang_encoder.config.hidden_size

        self.first_lang_encoder = first_lang_encoder
        self.ocr_lang_encoder = ocr_lang_encoder
        
        self.proj_img = nn.Linear(self.vis_dim, self.lang_dim)

        self.vision_encoder = vision_encoder
        self.perceiver = PerceiverResampler(dim=self.vis_dim)
        self.lang_encoder = lang_encoder
        self.lang_encoder.init_flamingo(
            media_token_id=media_token_id,
            lang_hidden_size=self.lang_dim,
            vis_hidden_size=self.vis_dim,
            cross_attn_every_n_layers=cross_attn_every_n_layers,
            gradient_checkpointing=gradient_checkpointing,
        )
        self.classifier = ClassificationHead(
            input_dim = self.lang_dim*1*5,
            inner_dim = self.lang_dim*1*5,
            num_classes = n_classes,
            pooler_dropout = 0.1,
            pooler_classifier = 'linear'
        )

        self._use_gradient_checkpointing = gradient_checkpointing
        self.perceiver._use_gradient_checkpointing = gradient_checkpointing
        # self.softmax = torch.nn.Softmax(dim=-1)

    def forward(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask = None,
        text_input_ids = None,
        text_attention_mask = None,
        ocr_input_ids = None,
        ocr_attention_mask = None,
        clear_conditioned_layers: bool = True,
        ):
        """
        Forward pass of Flamingo.

        Args:
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W) with F=1
            lang_x (torch.Tensor): Language input ids
                shape (B, T_txt)
            attention_mask (torch.Tensor, optional): Attention mask. Defaults to None.
            labels (torch.Tensor, optional): Labels. Defaults to None.
            clear_conditioned_layers: if True, clear the conditioned layers
                once the foward pass is completed. Set this to false if the
                same set of images will be reused in another subsequent
                forward pass.
            past_key_values: pre-computed values to pass to language model.
                See past_key_values documentation in Hugging Face
                CausalLM models.
            use_cache: whether to use cached key values. See use_cache
                documentation in Hugging Face CausalLM models.
        """
        assert (
            self.lang_encoder.initialized_flamingo
        ), "Flamingo layers are not initialized. Please call `init_flamingo` first."

        assert (
            self.lang_encoder._use_cached_vision_x or vision_x is not None
        ), "Must provide either vision_x or have precached media using cache_media()."

        self.lang_encoder._use_cached_vision_x = True
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=lang_x)

        first_text_out = self.first_lang_encoder(
            input_ids=text_input_ids,
            attention_mask = text_attention_mask,
            output_hidden_states=True
        )
        first_text_out = first_text_out.hidden_states
        first_text_out = torch.cat(first_text_out[-1:], dim=-1) # B, seq_len,lang_dim*4
        first_text_out = first_text_out[:,0,:] # B, 1, lang_dim*4

        ocr_text_out = self.ocr_lang_encoder(
            input_ids=ocr_input_ids,
            attention_mask = ocr_attention_mask,
            output_hidden_states=True
        )
        
        ocr_text_out = ocr_text_out.hidden_states
        ocr_text_out = torch.cat(ocr_text_out[-1:], dim=-1) # B, seq_len,lang_dim*4
        ocr_text_out = ocr_text_out[:,0,:] # B, 1, lang_dim*4

        first_vision_out = self.get_vision_x(vision_x) # B, 1, vis_dim
        first_vision_out = self.proj_img(first_vision_out) # B, 1, lang_dim

        output = self.lang_encoder(
            input_ids=lang_x,
            attention_mask = attention_mask,
            output_hidden_states=True
        )
        # output = output.pooler_output
        # output = self.classifier(output)

        hidden_states = output.hidden_states
        last_hidden_state = torch.cat(hidden_states[-1:], dim=-1) # B, 1 + seq_len, H*4
        text_features = last_hidden_state[:,0,:] # B, H*4
        visual_features = last_hidden_state[:,1,:] # B, H*4

        cls_and_visual = torch.cat([first_text_out, first_vision_out, ocr_text_out, text_features,visual_features], dim=1)
        output = self.classifier(cls_and_visual)

        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False
        self.uncache_media()
        # output = self.softmax(output)

        return output

    def get_vision_x(self, vision_x: torch.Tensor):
        assert vision_x.ndim == 6, "vision_x should be of shape (b, T_img, F, C, H, W)"
        b, T, F = vision_x.shape[:3]
        assert F == 1, "Only single frame supported"

        vision_x = rearrange(vision_x, "b T F c h w -> (b T F) c h w")
        with torch.no_grad():
            # vision_x = self.vision_encoder(vision_x)[1].unsqueeze(1) #check
#             vision_x = self.vision_encoder(vision_x)[1]
            out = self.vision_encoder(vision_x)[0]
        return out

    def _encode_vision_x(self, vision_x: torch.Tensor):
        """
        Compute media tokens from vision input by passing it through vision encoder and conditioning language model.
        Args:
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)

        rearrange code based on https://github.com/dhansmair/flamingo-mini
        """

        assert vision_x.ndim == 6, "vision_x should be of shape (b, T_img, F, C, H, W)"
        b, T, F = vision_x.shape[:3]
        assert F == 1, "Only single frame supported"

        vision_x = rearrange(vision_x, "b T F c h w -> (b T F) c h w")
        with torch.no_grad():
            # vision_x = self.vision_encoder(vision_x)[1].unsqueeze(1) #check
            vision_x = self.vision_encoder(vision_x)[1]

        vision_x = rearrange(vision_x, "(b T F) v d -> b T F v d", b=b, T=T, F=F)
        vision_x = self.perceiver(vision_x)
        # print('vision size:', vision_x.size())
        for layer in self.lang_encoder._get_encoder_layers():
            layer.condition_vis_x(vision_x)
        def clip_grad_norm_(max_norm):
            self.perceiver.clip_grad_norm_(max_norm)
            for layer in self.lang_encoder.gated_cross_attn_layers:
                if layer is not None:
                    layer.clip_grad_norm_(max_norm)
            self.lang_encoder.get_input_embeddings().clip_grad_norm_(max_norm)

        self.clip_grad_norm_ = clip_grad_norm_

    def _condition_media_locations(self, input_ids: torch.Tensor):
        """
        Compute the media token locations from lang_x and condition the language model on these.
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
        """
        media_locations = input_ids == self.media_token_id

        for layer in self.lang_encoder._get_encoder_layers():
            layer.condition_media_locations(media_locations)

    def cache_media(self, input_ids: torch.Tensor, vision_x: torch.Tensor):
        """
        Pre-cache a prompt/sequence of images / text for log-likelihood evaluations.
        All subsequent calls to forward() will generate attending to the LAST
        image in vision_x.
        This is not meant to be used to cache things for generate().
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)
        """
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=input_ids)
        self.lang_encoder._use_cached_vision_x = True

    def uncache_media(self):
        """
        Clear all conditioning.
        """
        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False

In [34]:
__KNOWN_encoder_layerS_ATTR_NAMES = {
    'roberta': 'encoder.layer',
    'MultilingualCLIP': 'transformer.encoder.layer',
    'ernie': 'encoder.layers'
}

In [35]:
def _infer_encoder_layers_attr_name(model):
    for k in __KNOWN_encoder_layerS_ATTR_NAMES:
        if k.lower() in model.__class__.__name__.lower():
            return __KNOWN_encoder_layerS_ATTR_NAMES[k]

    raise ValueError(
        f"We require the attribute name for the nn.ModuleList in the decoder storing the transformer block layers. Please supply this string manually."
    )


In [36]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel
from transformers import CLIPVisionModel, CLIPVisionConfig

import torch
from torchvision.io import read_image

In [37]:
lang_encoder = AutoModel.from_pretrained(PRETRAINED_PATH)
# vision_encoder = AutoModel.from_pretrained(VISION_PRETRAINED_PATH)
first_lang_encoder = AutoModel.from_pretrained(PRETRAINED_PATH)
ocr_lang_encoder = AutoModel.from_pretrained(PRETRAINED_PATH)

# first_lang_encoder.resize_token_embeddings(len(tokenizer))

temp_vision, _, _ = open_clip.create_model_and_transforms(
    'ViT-L-14-quickgelu',
    pretrained='dfn2b'
)

vision_encoder = temp_vision.visual
vision_encoder.output_tokens = True
vision_encoder.attn_pool = vision_encoder.proj = None


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [38]:
print("trainable parameter vision:", sum(p.numel() for p in vision_encoder.parameters()))
print("trainable parameter text:", sum(p.numel() for p in lang_encoder.parameters()))


trainable parameter vision: 303179776
trainable parameter text: 278042880


In [39]:
extend_instance(lang_encoder, FlamingoLMMixin)

In [40]:
encoder_layers_attr_name  = _infer_encoder_layers_attr_name(lang_encoder)
lang_encoder.set_encoder_layers_attr_name(encoder_layers_attr_name)
lang_encoder.resize_token_embeddings(len(tokenizer))


Embedding(250003, 768, padding_idx=1)

In [41]:
model = Flamingo(
    vision_encoder,
    lang_encoder,
    first_lang_encoder,
    ocr_lang_encoder,
    cross_attn_every_n_layers=1,
    vis_dim = 1024,
    media_token_id = tokenizer.convert_tokens_to_ids('<image>'),
    gradient_checkpointing=False,
    n_classes = 3
)

In [42]:
model.to(device)

Flamingo(
  (first_lang_encoder): ErnieMModel(
    (embeddings): ErnieMEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ErnieMEncoder(
      (layers): ModuleList(
        (0-11): 12 x ErnieMEncoderLayer(
          (self_attn): ErnieMAttention(
            (self_attn): ErnieMSelfAttention(
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (dro

In [43]:
print("trainable parameter text-vision:", sum(p.numel() for p in model.parameters()))

trainable parameter text-vision: 1273267203


In [44]:
# for data in tqdm(train_loader):
#     pass


In [129]:
for data in train_loader:
    break

with torch.no_grad():
    input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, label = data
    pixel_values = pixel_values.float()

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    text_input_ids = text_input_ids.to(device)
    text_attention_mask = text_attention_mask.to(device)
    ocr_input_ids = ocr_input_ids.to(device)
    ocr_attention_mask = ocr_attention_mask.to(device)

    pixel_values = pixel_values.to(device)
    label = label.to(device)

    logits = model(
            vision_x =  pixel_values,
            lang_x =  input_ids,
            attention_mask = attention_mask,
            text_input_ids = text_input_ids,
            text_attention_mask = text_attention_mask,
            ocr_input_ids = ocr_input_ids,
            ocr_attention_mask = ocr_attention_mask
            )
logits.size()

torch.Size([2, 3])

# TRAINING

In [130]:
import math
import torch
from torch.optim import Optimizer

class AdEMAMix(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999, 0.9999), eps=1e-8,
                 weight_decay=0, alpha=5.0, T_alpha_beta3=None):
        if not 0.0 <= lr:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        assert len(betas) == 3, f"Invalid beta parameters: {betas}, expected 3"
        assert all(0.0 <= beta < 1.0 for beta in betas), f"Invalid beta parameters: {betas}"
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        alpha=alpha, T_alpha_beta3=T_alpha_beta3)
        super(AdEMAMix, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdEMAMix, self).__setstate__(state)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            exp_avgs = []
            exp_avg_sqs = []
            exp_avg_slow = []
            state_steps = []

            for p in group['params']:
                if p.grad is not None:
                    params_with_grad.append(p)
                    if p.grad.is_sparse:
                        raise RuntimeError('AdEMAMix does not support sparse gradients')
                    grads.append(p.grad)

                    state = self.state[p]
                    # Lazy state initialization
                    if len(state) == 0:
                        state['step'] = 0
                        # Exponential moving average of gradient values
                        state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Exponential moving average of squared gradient values
                        state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Slow exponential moving average
                        state['exp_avg_slow'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                    exp_avgs.append(state['exp_avg'])
                    exp_avg_sqs.append(state['exp_avg_sq'])
                    exp_avg_slow.append(state['exp_avg_slow'])
                    state['step'] += 1
                    state_steps.append(state['step'])

            beta1, beta2, beta3 = group['betas']
            alpha = group['alpha']
            T_alpha_beta3 = group['T_alpha_beta3']

            self._update_adamemix(
                params_with_grad,
                grads,
                exp_avgs,
                exp_avg_sqs,
                exp_avg_slow,
                state_steps,
                beta1=beta1,
                beta2=beta2,
                beta3=beta3,
                alpha=alpha,
                T_alpha_beta3=T_alpha_beta3,
                lr=group['lr'],
                weight_decay=group['weight_decay'],
                eps=group['eps'],
            )

        return loss

    def _update_adamemix(self, params, grads, exp_avgs, exp_avg_sqs, exp_avg_slow, state_steps,
                         beta1, beta2, beta3, alpha, T_alpha_beta3, lr, weight_decay, eps):

        for i, param in enumerate(params):
            grad = grads[i]
            exp_avg = exp_avgs[i]
            exp_avg_sq = exp_avg_sqs[i]
            exp_avg_slow_i = exp_avg_slow[i]
            step = state_steps[i]

            bias_correction1 = 1 - beta1 ** step
            bias_correction2 = 1 - beta2 ** step

            if T_alpha_beta3 is not None:
                alpha_t = min(step * alpha / T_alpha_beta3, alpha)
                beta3_t = min(math.exp(math.log(beta1) * math.log(beta3) /
                              ((1 - step / T_alpha_beta3) * math.log(beta3) +
                               (step / T_alpha_beta3) * math.log(beta1))), beta3)
            else:
                alpha_t = alpha
                beta3_t = beta3

            # Decay the first and second moment running average coefficient
            exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
            exp_avg_slow_i.mul_(beta3_t).add_(grad, alpha=1 - beta3_t)

            denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(eps)

            step_size = lr / bias_correction1

            if weight_decay != 0:
                param.add_(param, alpha=-weight_decay * lr)

            param.addcdiv_(exp_avg + alpha_t * exp_avg_slow_i, denom, value=-step_size)

In [131]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters,
                        lr=3e-5)
for p in optimizer_grouped_parameters:
    p["lr"] = 3e-5

# optimizer = AdEMAMix(optimizer_grouped_parameters,
#                         lr=3e-5,weight_decay = 0.1, alpha=8.0)

In [132]:
import torch.nn.functional as F

def linear_combination(x, y, epsilon):
    return epsilon * x + (1 - epsilon) * y


def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction == 'mean' else loss.sum() if reduction == 'sum' else loss


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, weights, epsilon: float = 0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
        self.weights = weights
    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction,weight=self.weights)
        return linear_combination(loss / n, nll, self.epsilon)

In [133]:
weights

array([0.00026309, 0.00026309, 0.00026309, ..., 0.00251256, 0.01449275,
       0.00026309])

In [134]:
criterion = LabelSmoothingCrossEntropy(weights = torch.tensor(labels_weights).float().to('cuda'))
criterion(torch.randn(8,3).to('cuda'), torch.randint(0,3,(8,)).to('cuda'))

tensor(1.6760, device='cuda:0')

In [135]:
NUM_EPOCHS = 8

In [136]:
num_training_step = len(train_loader)*NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=num_training_step//10, num_training_steps=num_training_step)


In [137]:
tokenizer.convert_tokens_to_ids('<image>')

250002

In [138]:
from tqdm import tqdm

def train_one_epoch(model, criterion, dataloader, epoch,device,scaler):
    epoch_loss = 0

    model.train()

    # batch accumulation parameter
    accum_iter = 2

    with tqdm(dataloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, label = data
            pixel_values = pixel_values.float()
        
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            text_input_ids = text_input_ids.to(device)
            text_attention_mask = text_attention_mask.to(device)
            ocr_input_ids = ocr_input_ids.to(device)
            ocr_attention_mask = ocr_attention_mask.to(device)
        
            pixel_values = pixel_values.to(device)
            label = label.to(device)

            with torch.autocast(device_type='cuda', dtype=torch.bfloat16 if mode == 'BF16' else torch.float16, enabled=True if '16' in mode else False):
                logits = model(
                        vision_x =  pixel_values,
                        lang_x =  input_ids,
                        attention_mask = attention_mask,
                        text_input_ids = text_input_ids,
                        text_attention_mask = text_attention_mask,
                        ocr_input_ids = ocr_input_ids,
                        ocr_attention_mask = ocr_attention_mask
                        )

                losses = criterion(logits,label)

            lll = losses.item()
            epoch_loss += lll

            all_asp_loss = losses / accum_iter
            scaler.scale(all_asp_loss).backward()

            if ((i + 1) % accum_iter == 0) or (i + 1 == len(dataloader)):
                torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()


            tepoch.set_postfix(loss=lll)

        epoch_loss /= i
        print(f"At EPOCH {epoch}, loss = {epoch_loss}")
        print("======================================")

    return epoch_loss

In [139]:
from sklearn.metrics import precision_recall_fscore_support

def macro_f1(y_true, y_pred):
    p_macro, r_macro, f_macro, support_macro \
      = precision_recall_fscore_support(y_true, y_pred, average=None,zero_division=0.0,labels = [0,1,2,3])
    return p_macro, r_macro, f_macro


In [140]:
llabels = ['image-sarcasm','text-sarcasm','multi-sarcasm']

def eval(model, criterion, dataloader,device):

    print("EVAL STEP: ")

    model.eval()
    epoch_loss = 0
    preds = []
    labels = []
    with tqdm(dataloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, label = data
            pixel_values = pixel_values.float()
        
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            text_input_ids = text_input_ids.to(device)
            text_attention_mask = text_attention_mask.to(device)
            ocr_input_ids = ocr_input_ids.to(device)
            ocr_attention_mask = ocr_attention_mask.to(device)
        
            pixel_values = pixel_values.to(device)
            label = label.to(device)

            with torch.no_grad():
                logits = model(
                        vision_x =  pixel_values,
                        lang_x =  input_ids,
                        attention_mask = attention_mask,
                        text_input_ids = text_input_ids,
                        text_attention_mask = text_attention_mask,
                        ocr_input_ids = ocr_input_ids,
                        ocr_attention_mask = ocr_attention_mask
                        )

                eval_losses = criterion(logits,label)
                pred = np.argmax(logits.cpu().detach(),axis = -1).to('cpu').numpy()
                label = label.to('cpu').numpy()

                preds.append(pred)
                labels.append(label)

            epoch_loss += eval_losses

        epoch_loss /= i
        print(f"Loss = {epoch_loss}")

        preds = np.concatenate(preds)
        labels = np.concatenate(labels)
        precision, recall, f1_score = macro_f1(labels,preds)
        for idx, ll in enumerate(llabels):
            print(f"{ll}:")
            print("\t- Precision:", precision[idx])
            print("\t- Recall:", recall[idx])
            print("\t- F1-score:", f1_score[idx])
            print("========================")
        print("===============NEXT STEP===============")

        return epoch_loss

In [141]:
def save_model(path, model, epoch):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
    },path)

In [45]:
def load_model(path):
    check_point = torch.load(path,map_location = 'cpu')
    return check_point

In [143]:
best_loss = float('inf')
count = 0
for epoch in range(NUM_EPOCHS):
    train_one_epoch(model, criterion, train_loader, epoch, device,scaler)
    eval_loss = eval(model, criterion,dev_loader,device)
    if epoch >= NUM_EPOCHS - 3:
        save_model(f'model_{epoch}.pth',model, epoch)


Epoch 0:   0%|          | 1/2134 [00:00<15:08,  2.35batch/s, loss=0.814]/opt/conda/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

  warnings.warn(

Epoch 0: 100%|██████████| 2134/2134 [12:29<00:00,  2.85batch/s, loss=0.302]


At EPOCH 0, loss = 0.7594234634538352


EVAL STEP: 


Epoch 0: 100%|██████████| 238/238 [00:53<00:00,  4.42batch/s]


Loss = 1.3919011354446411

image-sarcasm:

	- Precision: 0.1245674740484429

	- Recall: 0.8181818181818182

	- F1-score: 0.21621621621621623


text-sarcasm:

	- Precision: 0.033783783783783786

	- Recall: 0.625

	- F1-score: 0.0641025641025641


multi-sarcasm:

	- Precision: 0.9473684210526315

	- Recall: 0.0851063829787234

	- F1-score: 0.1561822125813449


===============NEXT STEP===============


Epoch 1: 100%|██████████| 2134/2134 [12:30<00:00,  2.84batch/s, loss=0.369]


...

Loss = 0.691278874874115

image-sarcasm:

	- Precision: 0.42857142857142855

	- Recall: 0.4772727272727273

	- F1-score: 0.45161290322580644


text-sarcasm:

	- Precision: 0.5

	- Recall: 0.375

	- F1-score: 0.42857142857142855


multi-sarcasm:

	- Precision: 0.9333333333333333

	- Recall: 0.9267139479905437

	- F1-score: 0.930011862396204


===============NEXT STEP===============


Epoch 7: 100%|██████████| 2134/2134 [12:29<00:00,  2.85batch/s, loss=0.303]


At EPOCH 7, loss = 0.3346596301943273


EVAL STEP: 


Epoch 7: 100%|██████████| 238/238 [00:53<00:00,  4.44batch/s]


Loss = 0.6984819769859314

image-sarcasm:

	- Precision: 0.375

	- Recall: 0.4090909090909091

	- F1-score: 0.391304347826087


text-sarcasm:

	- Precision: 0.75

	- Recall: 0.375

	- F1-score: 0.5


multi-sarcasm:

	- Precision: 0.9290780141843972

	- Recall: 0.9290780141843972

	- F1-score: 0.9290780141843972


===============NEXT STEP===============


# MAKE RESULT

In [46]:
test_data = pd.read_json('/kaggle/input/dev-dsc2024/vimmsd-public-test.json',orient = 'index').reset_index()

In [47]:
normalize_class = TextNormalize()
test_data['caption'] = test_data['caption'].apply(lambda x: text_normalize(normalize_class.normalize(x)))

In [48]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)
tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<image>"]}
    )

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [49]:
# _, _, vision_processor = open_clip.create_model_and_transforms(
#     'ViT-L/14',
#     pretrained='openai'
# )

# PHASE 2

In [50]:
class TestDS_2(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, img_folder, vision_processor, max_len, ocr_text_dict):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.img_folder = img_folder
        self.vision_processor = vision_processor
        self.labels = ['image-sarcasm','text-sarcasm','multi-sarcasm']
        self.ocr_text_dict = ocr_text_dict

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        text = self.data.loc[idx,'caption']
        index = self.data.loc[idx,'index']
        
        img_name = self.data.loc[idx,'image']
        img_file = os.path.join(self.img_folder,img_name)

        ocr_text = self.ocr_text_dict[img_name.split('.')[0]]
        ocr_tokenizer = self.tokenizer(ocr_text, max_length=self.max_len,truncation='only_first',padding='max_length', return_length=True, return_tensors='pt')
        
        ocr_text = " ".join(ocr_text.split(" ")[:80])
        tokenizer_seq_dict = self.tokenizer(f"<image> {ocr_text}", text, max_length=self.max_len,truncation='longest_first',padding='max_length', return_length=True, return_tensors='pt')
        text_tokenizer = self.tokenizer(text, max_length=self.max_len,truncation='only_first',padding='max_length', return_length=True, return_tensors='pt')
        
        input_ids = tokenizer_seq_dict['input_ids'][0]
        attention_mask = (input_ids != 1).long()

        text_input_ids = text_tokenizer['input_ids'][0]
        text_attention_mask = (text_input_ids != 1).long()

        ocr_input_ids = ocr_tokenizer['input_ids'][0]
        ocr_attention_mask = (ocr_input_ids != 1).long()

        image = Image.open(img_file)
        pixel_values = self.vision_processor(image).unsqueeze(0).unsqueeze(0)

        return input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, index

In [52]:
test_data_2 = pd.read_json('/kaggle/input/phase-1-results/035_phase_1_results.json').reset_index()
test_data_2.shape

(1413, 3)

In [53]:
test_data_2 = test_data_2[test_data_2['results'] == 'sarcasm']
test_data_2.shape

(1003, 3)

In [54]:
test_data_2 = test_data[test_data['index'].isin(test_data_2['index'].values.tolist())].reset_index()
test_data_2

,level_0,index,image,caption,label
0,3,3,f5103b29053845e84baabbf4111b83fa9150f2aaa5e605...,có ai đang nghĩ giống tui hong 🤔 2 ảnh khoe áo...,NaN
1,9,9,14b9f8f0adce9a12db4b8382d0738576f01d0f904c3377...,có bao giờ bạn đánh mất những điều mà cứ ngỡ đ...,NaN
2,10,10,8de7e96685f312e075842c1f2c289a946d76ff6b438318...,lấy về sửa hay gì,NaN
3,11,11,c321cb796792a9a2f985ba7378eb589b3722025098dcfb...,- tụi mài có qánhh bài hông ? - dạ hông hông h...,NaN
4,12,12,13126175e7677c3cddaabac4286acb73eb2887c28da36b...,xưa và nay,NaN
...,...,...,...,...,...
998,1403,1403,37669d5ac043d72296733bbc04bc1d00c3efe991a158b9...,cre : internet,NaN
999,1404,1404,0ca7a313599afe2f5d8f4d3a145017e937607f886d86ee...,tất cả do bất đồng ngôn ngữ mà thôi : ( ( (,NaN
1000,1405,1405,d92052e80b4427045becbc8393024073a0b6e719c9db77...,lời anh hứa,NaN
1001,1406,1406,dce1f5cc3934875147313b5e859fce5cbb66e4807e2ff4...,tầm quan trọng của giáo dục zới tính : v,NaN


In [55]:
test_set = TestDS_2(test_data_2, tokenizer, '/kaggle/input/dev-dsc2024/public-test-images/dev-images', vision_processor, max_len = 256, ocr_text_dict=ocr_text_dict)

In [56]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle=True)

In [57]:
def load_model(path):
    check_point = torch.load(path,map_location = 'cpu')
    return check_point

In [58]:
checkpoint = load_model('/kaggle/input/all-weights/new_image_text_multi_7.pth')
model.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_30/1555856845.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  check_point = torch.load(path,map_location = 'cpu')


<All keys matched successfully>

In [59]:
labels = test_set.labels
labels

['image-sarcasm', 'text-sarcasm', 'multi-sarcasm']

In [60]:
from tqdm import tqdm
model.eval()
results = {}
for data in tqdm(test_loader):

    input_ids, attention_mask, text_input_ids, text_attention_mask, ocr_input_ids, ocr_attention_mask, pixel_values, indexes = data
    pixel_values = pixel_values.float()

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    text_input_ids = text_input_ids.to(device)
    text_attention_mask = text_attention_mask.to(device)
    ocr_input_ids = ocr_input_ids.to(device)
    ocr_attention_mask = ocr_attention_mask.to(device)
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        logits = model(
                vision_x =  pixel_values,
                lang_x =  input_ids,
                attention_mask = attention_mask,
                text_input_ids = text_input_ids,
                text_attention_mask = text_attention_mask,
                ocr_input_ids = ocr_input_ids,
                ocr_attention_mask = ocr_attention_mask
                )
        pred = np.argmax(logits.cpu().detach(),axis = -1).to('cpu').numpy()
        print(pred)
    for idx, pred in zip(indexes,pred):
        results[idx.item()] = labels[pred]

  3%|▎         | 1/32 [00:04<02:07,  4.11s/it]

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0]


  6%|▋         | 2/32 [00:07<01:49,  3.65s/it]

[2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2]


  9%|▉         | 3/32 [00:10<01:41,  3.50s/it]

...

[2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2]


 97%|█████████▋| 31/32 [01:43<00:03,  3.32s/it]

[2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2]


100%|██████████| 32/32 [01:44<00:00,  3.26s/it]

[2 2 2 2 0 2 2 2 2 2 2]


In [61]:
final_results = {
    'results': results,
    'phase': 'dev'
}
len(final_results['results'])

1003

In [62]:
final_results

{'results': {1247: 'multi-sarcasm',
  624: 'multi-sarcasm',
  1073: 'multi-sarcasm',
  1067: 'multi-sarcasm',
  308: 'multi-sarcasm',
  835: 'multi-sarcasm',
  1270: 'multi-sarcasm',
  228: 'multi-sarcasm',
  1129: 'multi-sarcasm',
  33: 'multi-sarcasm',
  1072: 'multi-sarcasm',
  1365: 'multi-sarcasm',
  1235: 'multi-sarcasm',
  455: 'multi-sarcasm',
  816: 'multi-sarcasm',
  1238: 'multi-sarcasm',
  886: 'multi-sarcasm',
  1052: 'multi-sarcasm',
  168: 'multi-sarcasm',
  172: 'multi-sarcasm',
  221: 'multi-sarcasm',
  713: 'multi-sarcasm',
  48: 'multi-sarcasm',
  36: 'image-sarcasm',
  1233: 'multi-sarcasm',
  676: 'multi-sarcasm',
  976: 'multi-sarcasm',
  896: 'multi-sarcasm',
  1187: 'multi-sarcasm',
  1177: 'multi-sarcasm',
  521: 'multi-sarcasm',
  791: 'image-sarcasm',
  1183: 'multi-sarcasm',
  1128: 'image-sarcasm',
  711: 'multi-sarcasm',
  1199: 'multi-sarcasm',
  523: 'multi-sarcasm',
  607: 'multi-sarcasm',
  1018: 'multi-sarcasm',
  1137: 'multi-sarcasm',
  273: 'multi-

In [63]:
with open('phase_2_results.json', 'w',encoding = 'utf-8') as f:
    json.dump(final_results, f, ensure_ascii = False, indent = 2)

In [64]:
with open('/kaggle/input/phase-1-results/04_phase_1_results.json', 'r',encoding = 'utf-8') as f:
    phase_1_results = json.load(f)

In [65]:
with open('phase_2_results.json', 'r',encoding = 'utf-8') as f:
    phase_2_results = json.load(f)

In [66]:
final = phase_1_results.copy()
len(final['results'].items())

1413

In [67]:
for k,v in phase_2_results['results'].items():
    final['results'][k] = v

In [68]:
len(final['results'].items())

1413

In [69]:
with open('04_e7_results.json', 'w',encoding = 'utf-8') as f:
    json.dump(final, f, ensure_ascii = False, indent = 2)

In [70]:
import json
with open("04_e7_results.json") as f:
    data = json.load(f)
data

{'results': {'870': 'multi-sarcasm',
  '1343': 'multi-sarcasm',
  '1127': 'multi-sarcasm',
  '100': 'not-sarcasm',
  '90': 'multi-sarcasm',
  '536': 'multi-sarcasm',
  '609': 'multi-sarcasm',
  '1205': 'multi-sarcasm',
  '1358': 'multi-sarcasm',
  '1366': 'multi-sarcasm',
  '1329': 'multi-sarcasm',
  '1359': 'multi-sarcasm',
  '455': 'multi-sarcasm',
  '489': 'not-sarcasm',
  '607': 'multi-sarcasm',
  '74': 'not-sarcasm',
  '844': 'multi-sarcasm',
  '326': 'multi-sarcasm',
  '1219': 'not-sarcasm',
  '304': 'not-sarcasm',
  '578': 'multi-sarcasm',
  '658': 'multi-sarcasm',
  '77': 'not-sarcasm',
  '45': 'not-sarcasm',
  '1106': 'not-sarcasm',
  '1132': 'multi-sarcasm',
  '498': 'not-sarcasm',
  '914': 'multi-sarcasm',
  '307': 'multi-sarcasm',
  '743': 'multi-sarcasm',
  '300': 'not-sarcasm',
  '1393': 'multi-sarcasm',
  '1000': 'multi-sarcasm',
  '1405': 'multi-sarcasm',
  '904': 'multi-sarcasm',
  '1247': 'multi-sarcasm',
  '930': 'multi-sarcasm',
  '159': 'multi-sarcasm',
  '1221': '

In [71]:
new_data = {}
for k,v in data['results'].items():
    new_data[int(k)] = v

new_data = dict(sorted(new_data.items()))
new_data

{0: 'not-sarcasm',
 1: 'not-sarcasm',
 2: 'not-sarcasm',
 3: 'multi-sarcasm',
 4: 'not-sarcasm',
 5: 'not-sarcasm',
 6: 'not-sarcasm',
 7: 'not-sarcasm',
 8: 'not-sarcasm',
 9: 'multi-sarcasm',
 10: 'multi-sarcasm',
 11: 'multi-sarcasm',
 12: 'multi-sarcasm',
 13: 'multi-sarcasm',
 14: 'multi-sarcasm',
 15: 'multi-sarcasm',
 16: 'not-sarcasm',
 17: 'not-sarcasm',
 18: 'not-sarcasm',
 19: 'not-sarcasm',
 20: 'not-sarcasm',
 21: 'multi-sarcasm',
 22: 'multi-sarcasm',
 23: 'not-sarcasm',
 24: 'multi-sarcasm',
 25: 'multi-sarcasm',
 26: 'not-sarcasm',
 27: 'not-sarcasm',
 28: 'not-sarcasm',
 29: 'not-sarcasm',
 30: 'multi-sarcasm',
 31: 'not-sarcasm',
 32: 'not-sarcasm',
 33: 'multi-sarcasm',
 34: 'multi-sarcasm',
 35: 'multi-sarcasm',
 36: 'image-sarcasm',
 37: 'multi-sarcasm',
 38: 'not-sarcasm',
 39: 'multi-sarcasm',
 40: 'not-sarcasm',
 41: 'not-sarcasm',
 42: 'not-sarcasm',
 43: 'not-sarcasm',
 44: 'not-sarcasm',
 45: 'not-sarcasm',
 46: 'multi-sarcasm',
 47: 'image-sarcasm',
 48: 'mu

In [72]:
results = {
    "results": new_data,
    "phase": 'dev'
}
results

{'results': {0: 'not-sarcasm',
  1: 'not-sarcasm',
  2: 'not-sarcasm',
  3: 'multi-sarcasm',
  4: 'not-sarcasm',
  5: 'not-sarcasm',
  6: 'not-sarcasm',
  7: 'not-sarcasm',
  8: 'not-sarcasm',
  9: 'multi-sarcasm',
  10: 'multi-sarcasm',
  11: 'multi-sarcasm',
  12: 'multi-sarcasm',
  13: 'multi-sarcasm',
  14: 'multi-sarcasm',
  15: 'multi-sarcasm',
  16: 'not-sarcasm',
  17: 'not-sarcasm',
  18: 'not-sarcasm',
  19: 'not-sarcasm',
  20: 'not-sarcasm',
  21: 'multi-sarcasm',
  22: 'multi-sarcasm',
  23: 'not-sarcasm',
  24: 'multi-sarcasm',
  25: 'multi-sarcasm',
  26: 'not-sarcasm',
  27: 'not-sarcasm',
  28: 'not-sarcasm',
  29: 'not-sarcasm',
  30: 'multi-sarcasm',
  31: 'not-sarcasm',
  32: 'not-sarcasm',
  33: 'multi-sarcasm',
  34: 'multi-sarcasm',
  35: 'multi-sarcasm',
  36: 'image-sarcasm',
  37: 'multi-sarcasm',
  38: 'not-sarcasm',
  39: 'multi-sarcasm',
  40: 'not-sarcasm',
  41: 'not-sarcasm',
  42: 'not-sarcasm',
  43: 'not-sarcasm',
  44: 'not-sarcasm',
  45: 'not-sarca

In [73]:
with open('04_e7_results.json', 'w',encoding = 'utf-8') as f:
    json.dump(results, f, ensure_ascii = False, indent = 2)